In [38]:
import os
from typing import Any, Literal, TypedDict

from dotenv import load_dotenv
from openai import AzureOpenAI
from pydantic import BaseModel, Field

In [15]:
load_dotenv()

client = AzureOpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [247]:
import json
from enum import Enum


class Role(Enum):
    SYSTEM = "system"
    USER = "user"
    ASSISTANT = "assistant"


class Message(TypedDict):
    role: Role
    content: str
    tool_call_id: str | None = Field(None)


sys_prompt = "You are a helpful assistant"
input_ = "how are you"
message_stack = [
    Message(role="system", content=sys_prompt),
    Message(role="user", content=input_),
]
send_request = client.chat.completions.create(messages=message_stack, model="gpt-4o")
response_body = json.loads(send_request.to_json())

In [11]:
response_body

{'id': 'chatcmpl-BU7yfUBPhxJyhpacIIDTTfCs7xP6C',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': "Thank you for asking! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with anything you need. How about you? How are you doing today?",
    'refusal': None,
    'role': 'assistant',
    'annotations': []},
   'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'},
    'self_harm': {'filtered': False, 'severity': 'safe'},
    'sexual': {'filtered': False, 'severity': 'safe'},
    'violence': {'filtered': False, 'severity': 'safe'}}}],
 'created': 1746520205,
 'model': 'gpt-4o-2024-11-20',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_ee1d74bde0',
 'usage': {'completion_tokens': 41,
  'prompt_tokens': 19,
  'total_tokens': 60,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_token

In [171]:
TOOL_TYPE = Literal["function"]


class LLMToolFunctionParameter(BaseModel):
    type: str = Field("object", description="function name")
    properties: dict[str, Any] | None = Field(None, description="function input schema")
    required: list[str] | None = Field(None, description="function input schema")


class LLMToolFunction(BaseModel):
    name: str = Field(..., description="function name")
    description: str = Field(..., description="function description")
    parameters: LLMToolFunctionParameter | None = Field(None)


class LLMTool(BaseModel):
    type: TOOL_TYPE = Field("function", description="custom toll")
    function: LLMToolFunction = Field(..., description="custom toll")
    # a:bool = Field(..., description="testing")
    # parameters:LLMToolFunctionParameter = Field(...)

    # @classmethod
    # def get_function_parameters(self) -> dict[str, Any]:
    #     """Convert the Pydantic model to a tool schema for OpenAI."""
    #     properties = {}
    #     required = []

    #     # Get fields from the model
    #     fields = self.model_fields

    #     for field_name, field in fields.items():
    #         if field_name in ['type','function']:
    #             continue

    #         # Get field type
    #         field_type = field.annotation

    #         if field_type is bool:
    #             properties[field_name] = {
    #                 "type": "boolean",
    #                 "description": field.description,
    #             }
    #         # Add to required if not optional and no default
    #         if field.is_required():  # and field.default is None:
    #             required.append(field_name)

    #     return properties, required


class TestTool(LLMTool):
    a: bool = Field(False, description="testing")

    # def __init__(self):
    def __init__(self, llm_tool: LLMTool):
        super().__init__(llm_tool)

        # self.function = LLMToolFunction.model_construct()
        # properties, required = self.LLMTool.get_function_parameters()

        # self.function.model_construct({"properties": properties, "required": required})

    # def model_post_init(context: Any) -> None:

    #     properties, required = self.LLMTool.get_function_parameters()
    #     self.function.model_cons|truct({"properties": properties, "required": required})

In [189]:
llm_function = LLMToolFunction(**{"name": "name", "description": "desc"})
l = LLMTool(function=llm_function)

In [182]:
llm_function.model_dump()

{'name': 'name', 'description': 'desc', 'parameters': None}

In [187]:
llm_function.model_dump()

{'name': 'name', 'description': 'desc', 'parameters': None}

In [190]:
tt = TestTool(llm_tool=l)

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [173]:
tt = TestTool(**{"function": llm_function, "a": True})

TypeError: TestTool.__init__() got an unexpected keyword argument 'function'

In [170]:
tt.model_dump()

{'type': 'function',
 'function': {'name': 'name', 'description': 'desc', 'parameters': None},
 'a': True}

In [158]:
tt = TestTool(function=llm_function, a=True)

TypeError: TestTool.model_post_init() takes 1 positional argument but 2 were given

In [133]:
l.get_function_parameters()

AttributeError: 'LLMTool' object has no attribute 'get_function_parameters'

In [145]:
l.model_dump()

{'type': 'function',
 'function': {'name': 'name', 'description': 'desc', 'parameters': None}}

In [135]:
type(l)

__main__.LLMTool

In [143]:
llm_function = LLMToolFunction({"name": "name", "description": "desc"})

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [142]:
llm_function = LLMToolFunction(name="name", description="desc")
l = LLMTool(function=llm_function)

tt = TestTool(l)

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [108]:
tt = TestTool(LLMTool(function=LLMToolFunction(name="name", description="desc")))

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [63]:
llm_function = LLMToolFunction(name="name", description="desc")
tt = TestTool(llm_tool=LLMTool(function=llm_function))

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [56]:
llm_function = LLMToolFunction(name="name", description="desc")
tt = TestTool(LLMTool(function=llm_function))

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [29]:
t = LLMTool(function={"name": "name", "description": "desc"})

In [43]:
tt = TestTool(LLMTool(function={"name": "name", "description": "desc"}))

ValidationError: 1 validation error for LLMTool
function.parameters
  Field required [type=missing, input_value={'name': 'name', 'description': 'desc'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [191]:
import functools
import inspect
import json
from collections.abc import Callable
from typing import Any, get_type_hints

from pydantic import BaseModel, create_model


class Tool:
    """Base class for LLM tools."""

    def __init__(
        self,
        name: str,
        description: str,
        parameters_model: type[BaseModel],
        function: Callable,
    ):
        self.name = name
        self.description = description
        self.parameters_model = parameters_model
        self.function = function

    def __call__(self, **kwargs):
        """Execute the tool with validated parameters."""
        # Validate paratool_call.function.argumentsmeters
        validated_params = self.parameters_model(**kwargs)
        # Call the function with validated parameters
        return self.function(**validated_params.model_dump())

    def to_openai_schema(self) -> dict[str, Any]:
        """Convert the tool to OpenAI's function calling format."""
        schema = self.parameters_model.model_json_schema()

        # OpenAI expects a specific format
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": schema,
            },
        }

    @classmethod
    def from_function(
        cls,
        function: Callable,
        name: str | None = None,
        description: str | None = None,
    ) -> "Tool":
        """Create a tool from a function with type hints."""
        if name is None:
            name = function.__name__

        if description is None:
            description = function.__doc__ or f"Tool for {name}"

        # Get type hints from the function
        hints = get_type_hints(function)
        signature = inspect.signature(function)

        # Create field definitions for Pydantic model
        fields = {}
        for param_name, param in signature.parameters.items():
            param_type = hints.get(param_name, Any)
            default = ... if param.default is inspect.Parameter.empty else param.default
            fields[param_name] = (param_type, default)

        # Create a Pydantic model for parameters
        parameters_model = create_model(f"{name.capitalize()}Parameters", **fields)

        return cls(name, description, parameters_model, function)


def create_tool(
    name: str, description: str, parameters_model: type[BaseModel]
) -> Callable[[Callable], Tool]:
    """Decorator to create a tool with a custom parameters model."""

    def decorator(func: Callable) -> Tool:
        tool = Tool(name, description, parameters_model, func)
        functools.update_wrapper(tool, func)
        return tool

    return decorator


class ToolRegistry:
    """Registry to keep track of all tools."""

    def __init__(self):
        self.tools: dict[str, Tool] = {}

    def register(self, tool: Tool) -> Tool:
        """Register a tool in the registry."""
        self.tools[tool.name] = tool
        return tool

    def get_tool(self, name: str) -> Tool | None:
        """Get a tool by name."""
        return self.tools.get(name)

    def list_tools(self) -> list[str]:
        """List all registered tool names."""
        return list(self.tools.keys())

    def get_openai_schemas(self) -> list[dict[str, Any]]:
        """Get OpenAI schemas for all registered tools."""
        return [tool.to_openai_schema() for tool in self.tools.values()]


# Create a global registry
registry = ToolRegistry()


def register_tool(
    name: str | None = None,
    description: str | None = None,
) -> Callable[[Callable], Tool]:
    """Decorator to create and register a tool from a function."""

    def decorator(func: Callable) -> Tool:
        tool = Tool.from_function(func, name, description)
        registry.register(tool)
        return tool

    return decorator

In [192]:
@register_tool(description="Get the weather forecast for a location")
def get_weather(location: str, days: int = 3) -> list[dict]:
    """Get weather forecast for the specified location.

    Args:
        location: City name or coordinates
        days: Number of days to forecast

    Returns:
        List of daily forecasts
    """
    # Actual implementation would call a weather API
    return [{"day": i, "temp": 70 + i, "condition": "Sunny"} for i in range(days)]

In [205]:
from datetime import date

from pydantic import BaseModel, Field


class BookSearchParams(BaseModel):
    query: str = Field(..., description="Search query for books")
    max_results: int = Field(10, description="Maximum number of results to return")
    published_after: date | None = Field(
        None, description="Only return books published after this date"
    )


@create_tool(
    name="search_books",
    description="Search for books in the library database",
    parameters_model=BookSearchParams,
)
def search_books(
    query: str, max_results: int = 10, published_after: date | None = None
) -> list[dict]:
    """Implementation of book search functionality"""
    # Actual implementation would search a database
    return [{"title": f"Book about {query}", "year": 2023}]


# Register the tool
registry.register(search_books)

In [238]:
class SearchStockPrice(BaseModel):
    name: str = Field(..., description="Search query for Stock name")


@create_tool(
    name="search_stock_price",
    description="Search stock price from the database",
    parameters_model=SearchStockPrice,
)
def search_stock_price(name: str) -> str | int:
    """Implementation of stock price search functionality"""
    if name in ["AAPL", "TSM"]:
        return 100
    else:
        return f"Stock {name} is not in our database."


# Register the tool
registry.register(search_stock_price)

In [259]:
sys_prompt = "You are a helpful assistant that is able to privide info about stocks"
input_ = "what is the stock price of AAPL?"

message_stack = [
    Message(role="system", content=sys_prompt),
    Message(role="user", content=input_),
]

llm_response1 = client.chat.completions.create(
    messages=message_stack,
    model="gpt-4o",
    tools=[search_stock_price.to_openai_schema()],
)

# message_stack.append(llm_response1.choices[0].message)
# response_body = json.loads(send_request.to_json())

In [261]:
message_stack.append(llm_response1.choices[0].message)
# response_body = json.loads(send_request.to_json())

In [262]:
for tool_call in llm_response1.choices[0].message.tool_calls:
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)

    # Get and execute the tool
    tool = registry.get_tool(tool_name)
    if tool:
        result = tool(**tool_args)
        print(f"Tool {tool_name} returned: {result}")

    message_stack.append(
        Message(role="tool", tool_call_id=tool_call.id, content=str(result))
    )

    #     {
    #     "role": "tool",
    #     "tool_call_id": tool_call.id,
    #     "content": str(result)
    # })

Tool search_stock_price returned: 100


In [263]:
message_stack

[{'role': 'system',
  'content': 'You are a helpful assistant that is able to privide info about stocks'},
 {'role': 'user', 'content': 'what is the stock price of AAPL?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oFw5sB1x3k9qMnrVOA1pMHjv', function=Function(arguments='{"name":"AAPL"}', name='search_stock_price'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_oFw5sB1x3k9qMnrVOA1pMHjv',
  'content': '100'}]

In [253]:
tool_call.id

'call_BzysvPH8QOfn9g11cBDMG9MU'

In [264]:
send_request = client.chat.completions.create(
    messages=message_stack,
    model="gpt-4o",
    tools=[search_stock_price.to_openai_schema()],
)
send_request.choices[0].message

ChatCompletionMessage(content='The stock price of AAPL is $100.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [235]:
tool_call.function.arguments

'{"name":"AAPL"}'

In [236]:
tool_call.function.name

'search_stock_price'

In [237]:
exec(tool_call.function.name(tool_call.function.arguments))

TypeError: 'str' object is not callable

In [225]:
search_stock_price(**{"name": "AAPL"})

100

In [220]:
send_request.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hop2U32mfJKIyKOSD7Vti5X3', function=Function(arguments='{"name":"AAPL"}', name='search_stock_price'), type='function')]), content_filter_results={})

In [214]:
response_body

{'id': 'chatcmpl-BUGYXfcmcP0c2B9lBXXH39xaQ7ff2',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'tool_calls': [{'id': 'call_hop2U32mfJKIyKOSD7Vti5X3',
      'function': {'arguments': '{"name":"AAPL"}',
       'name': 'search_stock_price'},
      'type': 'function'}]},
   'content_filter_results': {}}],
 'created': 1746553181,
 'model': 'gpt-4o-2024-11-20',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_ee1d74bde0',
 'usage': {'completion_tokens': 17,
  'prompt_tokens': 84,
  'total_tokens': 101,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'prompt_filter_results': [{'prompt_index': 0,
   'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'},
    '

In [196]:
registry.tools

{'get_weather': <__main__.Tool at 0x114f17020>,
 'search_books': <__main__.Tool at 0x114f16c60>}

In [200]:
registry.tools["get_weather"].to_openai_schema()

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get the weather forecast for a location',
  'parameters': {'properties': {'location': {'title': 'Location',
     'type': 'string'},
    'days': {'default': 3, 'title': 'Days', 'type': 'integer'}},
   'required': ['location'],
   'title': 'Get_weatherParameters',
   'type': 'object'}}}